# Part 1

In [1]:
import pandas as pd
households = pd.read_csv("lcfs_2019_dvhh_ukanon.tab", delimiter="\t")
persons = pd.read_csv("lcfs_2019_dvper_ukanon201920.tab", delimiter="\t")
spending = pd.read_csv("lcfs_2019_dv_set89_ukanon.tab", delimiter="\t")
#spending is weekly

/Users/inesfernandez/opt/anaconda3/envs/openfisca/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (1872,1875) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#Codes for larger categories
categories = {"Food and non-alcoholic beverages": "1.", 
"Alcoholic beverages, tobacco and narcotics": "2.",
"Clothing and footwear": "3.",
"Housing, water, electricity, gas and other fuels": "4.", 
"Furnishings, household equipment and routine maintenance of house": "5.",
"Health": "6.",
"Transport": "7.",
"Communication": "8.",
"Recreation and culture": "9.",
"Education": "10",
"Restaurants and hotels": "11",
"Miscellaneous goods and services": "12",
#"Non-consumption expenditure": "20"
             }

In [3]:
spending["COI_PLUS"] = spending["COI_PLUS"].transform(lambda x: x[:2])

#if code is 20 change to 12 to include non-consumption expenditure in misc goods & services
spending["COI_PLUS"].replace({"20": "12"}, inplace=True)

In [4]:
#Total expenditure for each category
y = spending.groupby(["COI_PLUS"]).pdamount.sum()
total_expenditure = {}
for n in categories.keys():
    value = y.loc[categories[n]] * 52 #because weekly
    total_expenditure[n] = value

In [5]:
#Total emissions grouped by NCFS categories in Ktonnes
emissions = pd.read_csv("emissions.csv")

In [6]:
#NCFS categories included in each LCFS larger category, excluding non-consumption expenditure
categories_emissions = {'Food and non-alcoholic beverages': ['Food', 'Non-alcoholic beverages'],
 'Alcoholic beverages, tobacco and narcotics': ['Alcoholic beverages', 'Tobacco'],
 'Clothing and footwear': ['Clothing', 'Footwear'],
 'Housing, water, electricity, gas and other fuels': [
     #'Actual rentals for households',
       'Imputed rentals for households', 
     'Electricity, gas and other fuels'],             
 'Furnishings, household equipment and routine maintenance of house': ['Furniture, furnishings, carpets etc', 'Household textiles',
       'Household appliances', 'Glassware, tableware and household utensils',
       'Tools and equipment for house and garden',
       'Goods and services for household maintenance'],
 'Health': ['Medical products, appliances and equipment', 'Hospital services'],
 'Transport': ['Purchase of vehicles', 'Operation of personal transport equipment',
       'Transport services'],
 'Communication': ['Postal services',
       'Telephone and telefax equipment', 'Telephone and telefax services'],
 'Recreation and culture': ['Audio-visual, photo and info processing equipment',
       'Other major durables for recreation and culture',
       'Other recreational equipment etc',
       'Recreational and cultural services','Newspapers, books and stationery'],
 'Education': ['Education'],
 'Restaurants and hotels': ['Restaurants and hotels'],
 'Miscellaneous goods and services': ['Miscellaneous goods and services']}

In [7]:
#Total emissions for each LCFS larger category
emissions_per_cat = {}
for n in categories_emissions.keys():
    items = categories_emissions[n]
    total = 0
    for m in items:
        total = total + float(emissions[m][0])
    #since units is in ktonnes
    emissions_per_cat[n] = total * 10000

#Emissions per pound spent (in tonnes)
emissions_per_pound = {}
for category in categories_emissions.keys():
    emissions_per_pound[category] = total_expenditure[category]/emissions_per_cat[category]

emissions_per_pound 

{'Food and non-alcoholic beverages': 0.08174269929055596,
 'Alcoholic beverages, tobacco and narcotics': 0.4240110592776958,
 'Clothing and footwear': 0.0839986723262802,
 'Housing, water, electricity, gas and other fuels': 0.0006095434072255929,
 'Furnishings, household equipment and routine maintenance of house': 0.04265684229443317,
 'Health': 0.030199054674513247,
 'Transport': 0.008417136427727762,
 'Communication': 0.024907790005791648,
 'Recreation and culture': 0.05236182571465965,
 'Education': 0.0060888995104243305,
 'Restaurants and hotels': 0.05437523790446789,
 'Miscellaneous goods and services': 0.09849971884573334}

In [25]:
#Emissions for a household
def get_category_total(household, category):
    cat_code = categories.get(category)
    household_rows = spending.loc[spending["case"] == household]
    household_rows["COI_PLUS"] = household_rows["COI_PLUS"].transform(lambda x: x[:2])
    
    #if code is 20 change to 12; include non-consumption in misc goods & services
    household_rows["COI_PLUS"].replace({"20": "12"}, inplace=True)
    
    relevant_rows = household_rows.loc[household_rows["COI_PLUS"] == cat_code]
    
    #multiply by 52 to get annual
    expenditure = relevant_rows["pdamount"].sum() * 52
    return expenditure * emissions_per_pound[category]

def get_household_total(household):
    emissions = 0
    for n in categories:
        category_emissions = (get_category_total(household, n))
        emissions = emissions + category_emissions
    return emissions

In [8]:
#This is a simpler version of the above, but gets an error with 4th category
x = spending.groupby(["case", "COI_PLUS"]).pdamount.sum()
def get_h_emissions(household):
    emits = 0
    for n in categories:
        category_expenditure = x[household].loc[categories[n]] * 52 #because weekly
        print(category_expenditure)
        emit = category_expenditure * emissions_per_pound[n]
        emits = emits + emit
    return emits

In [27]:
#Example
get_household_total(780)

2682.831561185424

In [31]:
#Add emissions column to households dataframe
households["Emissions"] = pd.Series(get_household_total(n) for n in households["case"])

# Part 2

In [48]:
#LCFS variables using field names
def get_LCFS_variables():
    from openfisca_uk.entities import Person, BenUnit, Household
    
    #Household size
    class A049(Variable):
        value_type = float
        entity = Household
        definition_period = ETERNITY
    
    #Gross household income (weekly)
    class P352p(Variable):
        value_type = float
        entity = Household
        definition_period = YEAR 
    
    #Equivalized income (McClements Scale)
    class EqIncDMp(Variable):
        value_type = float
        entity = Household
        definition_period = YEAR 
        
    #Equivalised income (OECD Scale)
    class EqIncDOp(Variable):
        value_type = float
        entity = Household
        definition_period = YEAR 
        
    #Location (gov office region)
    class Gorx(Variable):
        value_type = float
        entity = Household
        definition_period = ETERNITY 
        
    #Rent
    class B010(Variable):
        value_type = float
        entity = Household
        definition_period = ETERNITY

In [49]:
def get_input_variables():
    
    #Household size
    class household_size(Variable):
        value_type = float
        entity = Household
        label = "Number of people in household"
        definition_period = ETERNITY

        def formula(household, period, parameters):
            return household("A049", period) * WEEKS_IN_YEAR
    
    #Gross household income
    class gross_income(Variable):
        value_type = float
        entity = Household
        label = "Gross household income"
        definition_period = YEAR

        def formula(household, period, parameters):
            return household("P352p", period) * WEEKS_IN_YEAR
    
    #Equivalized income (McClements Scale)
    class equivalized_income(Variable):
        value_type = float
        entity = Household
        definition_period = YEAR 
        
        def formula(household, period, parameters):
            return household("EqIncDMp", period) * WEEKS_IN_YEAR 
            #if OECD scale, replace with "EqIncDOp"
        
    class region(Variable):
        value_type = Enum
        possible_values = Region
        default_value = Region.UNKNOWN
        entity = Household
        label = "Region of the UK"
        definition_period = ETERNITY

        def formula(household, period, parameters):
            region = household("Gorx", period)
            reg = select(
                [
                    region == 1,
                    region == 2,
                    region == 4,
                    region == 5,
                    region == 6,
                    region == 7,
                    region == 8,
                    region == 9,
                    region == 10,
                    region == 11,
                    region == 12,
                    region == 13,
                ],
                [
                    Region.NORTH_EAST,
                    Region.NORTH_WEST,
                    Region.YORKSHIRE,
                    Region.EAST_MIDLANDS,
                    Region.WEST_MIDLANDS,
                    Region.EAST_OF_ENGLAND,
                    Region.LONDON,
                    Region.SOUTH_EAST,
                    Region.SOUTH_WEST,
                    Region.WALES,
                    Region.SCOTLAND,
                    Region.NORTHERN_IRELAND,
                ],
            )
            return reg
    
    class rent(Variable):
        value_type = float
        entity = Household
        label = "Gross rent for the household"
        definition_period = YEAR

        def formula(household, period, parameters):
            return household("B010", period) * WEEKS_IN_YEAR

In [52]:
from openfisca_uk_data.datasets.frs.raw_frs import RawFRS
from pathlib import Path
from typing import List
from openfisca_core.model_api import *
from openfisca_uk_data.utils import dataset
import pandas as pd
import shutil
from openfisca_uk_data.utils import (
    CAPITAL_INCOME_VARIABLES,
    LABOUR_INCOME_VARIABLES,
    uprated,
)
import h5py
from openfisca_uk_data.datasets.frs.base_frs.dataset import BaseFRS
from openfisca_uk_data.datasets.frs.base_frs.model_input_variables import (
    get_input_variables,
)


def from_FRS(year: int = 2018):
    from openfisca_uk import CountryTaxBenefitSystem

    system = CountryTaxBenefitSystem()
    variables = []
    for variable in get_input_variables():
        try:
            variables += [type(system.variables[variable.__name__])]
        except:
            variables += [variable]
    for i in range(len(variables)):
        variable = variables[i]
        if variable.__name__ in LABOUR_INCOME_VARIABLES:
            variables[i] = uprated(
                "uprating.labour_income", from_year=year + 1
            )(variable)
        elif variable.__name__ in CAPITAL_INCOME_VARIABLES:
            variables[i] = uprated(
                "uprating.labour_income", from_year=year + 1
            )(variable)
        else:
            variables[i] = uprated(from_year=year + 1)(variable)

    class reform(Reform):
        def apply(self):
            for var in variables:
                self.update_variable(var)

    return reform


@dataset
class FRS:
    name = "frs"
    openfisca_uk_compatible = True
    input_reform_from_year = from_FRS

    def generate(year) -> None:
        base_frs_years = BaseFRS().years
        if len(base_frs_years) == 0:
            raw_frs_years = RawFRS().years
            if len(raw_frs_years) == 0:
                raise Exception("No FRS microdata to generate from")
            else:
                base_frs_year = max(raw_frs_years)
        else:
            base_frs_year = max(base_frs_years)
        from openfisca_uk import Microsimulation

        base_frs_sim = Microsimulation(dataset=BaseFRS, year=base_frs_year)
        person_vars, benunit_vars, household_vars = [
            [
                var.__name__
                for var in get_input_variables()
                if var.entity.key == entity
            ]
            for entity in ("person", "benunit", "household")
        ]
        with h5py.File(FRS.file(year), mode="w") as f:
            for variable in person_vars + benunit_vars + household_vars:
                f[f"{variable}/{year}"] = base_frs_sim.calc(
                    variable, year
                ).values

In [54]:
from openfisca_uk_data.utils import (
    MAIN_INPUT_VARIABLES,
    dataset,
    uprate_variables,
)
import synthimpute as si
import numpy as np
import h5py

class FRS_LCFS_Adjusted:
    name = "frs_lcfs_adj"
    openfisca_uk_compatible = True
    input_reform_from_year = uprate_variables(MAIN_INPUT_VARIABLES)

    def generate(year):
        from openfisca_uk import Microsimulation
        from openfisca_uk_data.datasets.frs.frs import FRS
        LCFS = households
        

        frs_sim = Microsimulation(dataset=FRS)
        lcfs_sim = Microsimulation(dataset=LCFS)
        
        #common variables
        lcfs_common_variables = np.array(
            [
                lcfs_sim.calc("household_size", year).values,
                lcfs_sim.calc("gross_household_income", year).values,
                lcfs_sim.calc("equivalized_income", year).values,
                lcfs_sim.calc("region", year).values,
                lcfs_sim.calc("rent", year).values,
                
            ]
        ).T
        
        frs_common_variables = np.array(
            [
                frs_sim.calc("household_size", year).values,
                frs_sim.calc("gross_household_income", year).values,
                frs_sim.calc("equivalized_income", year).values,
                frs_sim.calc("region", year).values,
                frs_sim.calc("rent", year).values,
            ]
        ).T
        
        #imputed variables
        lcfs_emissions = lcfs_sim.calc("dividend_income", year).values
        frs_weight = frs_sim.calc("household_weight", year).values
        
        print(
            "Imputing emissions for FRS respondents from LCFS values...",
            end="",
        )
        imputed_emissions = si.rf_impute(
            x_train=lcfs_common_variables,
            y_train=lcfs_emissions,
            x_new=frs_common_variables,
            sample_weight_train=frs_weight,
            mean_quantile=0.18,
        )
        print(" completed.")
        imputed_emissions *= (
            frs_sim.calc("emissions", year).values > 0
        )
        frs_sim.simulation.set_input(
            "emissions", year, imputed_emissions
        )
        with h5py.File(
            FRS_SPI_Adjusted.data_dir / FRS_SPI_Adjusted.filename(year), "w"
        ) as f:
            for variable in MAIN_INPUT_VARIABLES:
                f[f"{variable}/{year}"] = frs_sim.calc(variable, year).values
